参考ページ：https://cream-kuchen.hatenablog.com/entry/efficient_portfolio_pyportfolioopt

In [2]:
import pandas as pd
from datetime import datetime

# 投資信託のデータを取得する関数
def get_data(ISIN, FUND):
    """
    引数
    ISIN ... ファンドのISINコード (ファンド検索して[ISIN]を探す: https://www.jasdec.com/reading/itmei.php)
    FUND ... ファンドコード (ファンド検索して[ファンドの特色]右上を探す: https://www.morningstar.co.jp/FundData/DetailSearchResult.do?mode=1)
    戻値
    df ... 基準価額や純資産総額のpandasデータフレーム
    """
    # csvダウンロードするURLを作成し読み込む
    BASEURL = "https://toushin-lib.fwg.ne.jp/FdsWeb/FDST030000/csv-file-download?"
    ISINcd  = "isinCd="+ISIN
    FUNDcd  = "associFundCd="+FUND
    DOWNURL = BASEURL+ISINcd+"&"+FUNDcd
    # 日付カラムを加工してデータ取得
    DATE_PARSE = lambda date: datetime.strptime(date, "%Y年%m月%d日")
    df = pd.read_csv(DOWNURL, engine="python", encoding="shift-jis", index_col="年月日", parse_dates=True, date_parser=DATE_PARSE)
    return df

In [3]:
# 8つの資産クラスのデータ取得
df_jpn_stck = get_data(ISIN="JP90C000BRT6", FUND="29312154")  # 国内株式     (ニッセイTOPIX)
df_jpn_bond = get_data(ISIN="JP90C000B9V6", FUND="29314151")  # 国内債券     (ニッセイ内債)
df_jpn_reit = get_data(ISIN="JP90C00099G5", FUND="29314136")  # 国内リート   (ニッセイJリート)
df_dvp_stck = get_data(ISIN="JP90C0009VE0", FUND="2931113C")  # 先進国株式   (ニッセイ外株インデックス)
df_dvp_bond = get_data(ISIN="JP90C0009VF7", FUND="2931213C")  # 先進国債券   (ニッセイ先進国債券)
df_dvp_reit = get_data(ISIN="JP90C0009VG5", FUND="2931313C")  # 先進国リート (ニッセイグローバルリート)
df_emg_stck = get_data(ISIN="JP90C0006LK4", FUND="0331809A")  # 新興国株式   (MUFJ eMAXIS インデックス)
df_emg_bond = get_data(ISIN="JP90C000DVU2", FUND="0431U169")  # 新興国債券   (大和 iFree 新興国債券)
df_world_stck = get_data(ISIN="JP90C000H1T1", FUND="0331418A")  # 全世界株式  (emaxis slim 全世界株式 オールカントリー)

In [4]:
# 取得データを設定期間だけ抽出し結合する関数
def join_data(df_part, df_join, KEYWORD, str_date, end_date):
    """
    引数
    df_part ... 各資産クラスのデータフレーム
    df_join ... 複数の資産クラスを結合したデータフレーム
    KEYWORD ... 資産クラス名
    str_date ... 抽出期間の開始日
    end_date ... 抽出期間の終了日
    戻値
    df_join ... 複数の資産クラスを結合したデータフレーム
    """
    # 各資産クラスのデータフレームを期間抽出
    df_part_fil = df_part.loc[(df_part.index>=str_date)&(df_part.index<=end_date), :]
    df_part_fil = df_part_fil.rename(columns={"基準価額(円)":KEYWORD})[[KEYWORD]]
    # 複数の資産クラスを結合
    df_join = pd.merge(df_join, df_part_fil, left_index=True, right_index=True, how="inner") if df_join is not None else df_part_fil
    return df_join

In [5]:
# 資産クラス名とデータフレームの変数を組み合わせる
dict_assets = {
    "国内株式"     : df_jpn_stck,
    "国内債券"     : df_jpn_bond,
    "国内リート"   : df_jpn_reit,
    "先進国株式"   : df_dvp_stck,
    "先進国債券"   : df_dvp_bond,
    "先進国リート" : df_dvp_reit,
    "新興国株式"   : df_emg_stck,
    "新興国債券"   : df_emg_bond,
    "全世界株式"   : df_world_stck
}

# 取得データを直近過去5年間だけ抽出し結合
# (※注:  データの日付が抽出期間未満の場合、結合データも少なくなる)
df_join = None
str_date = "2001-04-01"  # 抽出開始日
end_date = "2022-06-30"  # 抽出終了日
# 資産クラス名とデータフレームの変数の組み合わせを横方向に結合
for KEYWORD, df_part in dict_assets.items():
    df_join = join_data(df_part, df_join, KEYWORD, str_date, end_date)

df_join.to_csv('assets_class.csv')